In [1]:
import re
import codecs
import json
import traceback
from urllib.parse import quote, unquote
from collections import defaultdict
from operator import itemgetter

from pyquery import PyQuery as pq
import requests


index_regex = re.compile(r'[^{]+(.*)\)')

In [17]:
# get brand image

resp = requests.get('https://item.tuhu.cn/Car/GetCarBrands2?callback=__GetCarBrands__&_=1565141557927')
index = json.loads(index_regex.findall(resp.text)[0])

In [21]:
# download logo image
for _, l1 in index.items():
    for brand_dict in l1:
        bname = brand_dict['Brand']
        bimg = requests.get(f"https://img3.tuhu.org/{brand_dict['Url']}").content
        with open(f'{bname}.png', 'wb') as f:
            f.write(bimg)
        

In [46]:
# get child tier

def get_child_brand_url(brand):
    return f"https://item.tuhu.cn/Car/SelOneBrand?callback=__GetCarBrands__&Brand={quote(brand)}&_=1565143202617"


result = defaultdict(list)
for _, l1 in index.items():
    for brand_dict in l1:
        bname = brand_dict['Brand']
        for cbrand_d in json.loads(index_regex.findall(requests.get(get_child_brand_url(bname)).text)[0])['OneBrand']:
            result[bname].append(cbrand_d)
                   

In [47]:
# write result

with open('./车胎/child-tires-utf8.txt', 'rb') as f:
    f.write(json.dumps(result, indent=4, sort_keys=True, ensure_ascii=False).encode('utf8'))
    
with open('./车胎/child-tires-gbk.txt', 'rb') as f:
    f.write(json.dumps(result, indent=4, sort_keys=True, ensure_ascii=False).encode('gbk')) 

In [3]:
# load result
with open('./车胎/child-tires-utf8.txt', 'rb') as f:
    result = json.loads(f.read().decode('utf8'))

In [48]:
index

{'A': [{'Brand': 'A - 阿尔法·罗米欧',
   'Url': '/Images/Logo/aerfa%C2%B7luomiou.png',
   'ImageUrl': None},
  {'Brand': 'A - 安驰', 'Url': '/Images/Logo/anchi.png', 'ImageUrl': None},
  {'Brand': 'A - ARCFOX', 'Url': '/Images/Logo/arcfox.png', 'ImageUrl': None},
  {'Brand': 'A - 阿斯顿·马丁',
   'Url': '/Images/Logo/asidun%C2%B7mading.png',
   'ImageUrl': None},
  {'Brand': 'A - 奥迪', 'Url': '/Images/Logo/aodi.png', 'ImageUrl': None}],
 'B': [{'Brand': 'B - 别克', 'Url': '/Images/Logo/bieke.png', 'ImageUrl': None},
  {'Brand': 'B - 奔驰', 'Url': '/Images/Logo/benchi.png', 'ImageUrl': None},
  {'Brand': 'B - 保时捷', 'Url': '/Images/Logo/baoshijie.png', 'ImageUrl': None},
  {'Brand': 'B - 宝龙', 'Url': '/Images/Logo/baolong.png', 'ImageUrl': None},
  {'Brand': 'B - 比亚迪', 'Url': '/Images/Logo/biyadi.png', 'ImageUrl': None},
  {'Brand': 'B - 宝马', 'Url': '/Images/Logo/baoma.png', 'ImageUrl': None},
  {'Brand': 'B - 标致', 'Url': '/Images/Logo/biaozhi.png', 'ImageUrl': None},
  {'Brand': 'B - 宾利', 'Url': '/Images/

In [74]:
# get child brand
from typing import List

headers = {
    'User-Agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 10_3_1 like Mac OS X) AppleWebKit/603.1.30 (KHTML, like Gecko) Version/10.0 Mobile/14E304 Safari/602.1',
}


def load_pailiang(vid) -> List[str]:
    url = f'https://api.tuhu.cn/Vehicle/SelectVehicle?VehicleId={vid}'
    return requests.get(url, headers=headers).json()['PaiLiang']
#     print(url, r)
#     return r['PaiLiang']


def load_brand_yrs(vid, pailiang) -> List[dict]:
    """
    Args:
    
        vid(str): 'VE-LSG-GL6'
        pailiang(str): '1.3T'
        
    Return: [{Key: "VE-LSG-GL6", Value: "2019"}, {Key: "VE-LSG-GL6", Value: "2018"},…]
    """
    url = f'https://item.tuhu.cn/Car/SelectVehicle?callback=__GetCarBrands__&VehicleID={vid}&PaiLiang={pailiang}&_=1565330440229'
    return json.loads(index_regex.findall(requests.get(url).text)[0])['Nian']


def load_child_brands(vid, pailiang, nian):
    """
    Args:
    
        nian(str): 2017
        
    Return: [{LiYangID: null, TID: "113193", Name: "2018款 1.3T 手自一体 18T 6座 豪华版"},…]
    """
    url = f'https://item.tuhu.cn/Car/SelectVehicleSalesName?callback=__GetCarBrands__&VehicleID={vid}&PaiLiang={pailiang}&Nian={nian}&_=1565330440230'
    return json.loads(index_regex.findall(requests.get(url).text)[0])['SalesName']



vid_dones = set([])

with open('./车系/child-brands-gbk.txt', '+ab') as gbk_fp, \
        open('./车系/child-brands-utf.txt', '+ab') as utf8_fp:
#         open('./车系/child-brands-utf-2.txt', '+ab') as utf8_fp_2, \
#         open('./车系/child-brands-gbk-2.txt', '+ab') as gbk_fp_2:
        
    utf8_fp.seek(0, 0)
    for ij in utf8_fp:
        vid_dones.add(json.loads(ij.decode('utf8'))['vid'])
        
    for b_1, bl in result.items():
        print(f'{b_1} done')
        for bd in bl:
            vid = bd['ProductID']
            if vid in vid_dones:
                continue            

            vid_dones.add(vid)
            try:
                for pailiang in load_pailiang(vid):
                    for nian in [d['Value'] for d in load_brand_yrs(vid, pailiang)]:
                        for b in load_child_brands(vid, pailiang, nian):
                            item = {'brand': b, 'pailiang': pailiang, 'vid': vid, 'year': nian}
                            ij = json.dumps(item, sort_keys=True, ensure_ascii=False) + '\n'
                            gbk_fp.write(ij.encode('gbk'))
                            utf8_fp.write(ij.encode('utf8'))
            except Exception:
                print(f'>> {b_1} - {vid} - {bd}')
                traceback.print_exc()
            finally:
                gbk_fp.flush()
                utf8_fp.flush()

                


A - 阿尔法·罗米欧 done
A - 安驰 done
A - ARCFOX done
A - 阿斯顿·马丁 done
A - 奥迪 done
B - 别克 done
B - 奔驰 done
B - 保时捷 done
B - 宝龙 done
B - 比亚迪 done
B - 宝马 done
B - 标致 done
>> B - 标致 - VE-GPEUG505 - {'Vehicle': '505-广州标致', 'ProductID': 'VE-GPEUG505', 'BrandType': '广州标致', 'Brand': 'B - 标致', 'Url': '/Images/Logo/biaozhi.png', 'Src': 'https://img3.tuhu.org/Images/type/VE-GPEUG505.png@163w_163h_100Q.png', 'IsBaoYang': 'True', 'CarName': '505', 'appSrc': None, 'Image': 'https://img3.tuhu.org/Images/type/VE-GPEUG505.png@100Q.png', 'ImageUrl': 'https://img3.tuhu.org/Images/Logo/BiaoZhi.png', 'Tires': '185/80R14', 'SpecialTireSize': None, 'OriginalIsBaoyang': 1, 'Priority2': 2, 'Priority3': 1, 'VehicleSeries': None}


Traceback (most recent call last):
  File "<ipython-input-74-a1bbc79ad05d>", line 60, in <module>
    for pailiang in load_pailiang(vid):
  File "<ipython-input-74-a1bbc79ad05d>", line 11, in load_pailiang
    return requests.get(url, headers=headers).json()['PaiLiang']
KeyError: 'PaiLiang'


B - 宾利 done
>> B - 宾利 - VE-BENT07AB - {'Vehicle': '雅骏-宾利汽车', 'ProductID': 'VE-BENT07AB', 'BrandType': '宾利汽车', 'Brand': 'B - 宾利', 'Url': '/Images/Logo/binli.png', 'Src': 'https://img3.tuhu.org/Images/type/VE-BENT07AB.png@163w_163h_100Q.png', 'IsBaoYang': 'True', 'CarName': '雅骏', 'appSrc': None, 'Image': 'https://img3.tuhu.org/Images/type/VE-BENT07AB.png@100Q.png', 'ImageUrl': 'https://img3.tuhu.org/Images/Logo/BinLi.png', 'Tires': '255/55R17', 'SpecialTireSize': None, 'OriginalIsBaoyang': 1, 'Priority2': 1, 'Priority3': 5, 'VehicleSeries': None}


Traceback (most recent call last):
  File "<ipython-input-74-a1bbc79ad05d>", line 60, in <module>
    for pailiang in load_pailiang(vid):
  File "<ipython-input-74-a1bbc79ad05d>", line 11, in load_pailiang
    return requests.get(url, headers=headers).json()['PaiLiang']
KeyError: 'PaiLiang'


B - 北汽制造 done
>> B - 北汽制造 - VE-LNB-LULING - {'Vehicle': '陆铃-北汽制造', 'ProductID': 'VE-LNB-LULING', 'BrandType': '北汽制造', 'Brand': 'B - 北汽制造', 'Url': '/Images/Logo/beiqizhizao.png', 'Src': 'https://img3.tuhu.org/Images/type/VE-LNB-LULING.png@163w_163h_100Q.png', 'IsBaoYang': 'True', 'CarName': '陆铃', 'appSrc': None, 'Image': 'https://img3.tuhu.org/Images/type/VE-LNB-LULING.png@100Q.png', 'ImageUrl': 'https://img3.tuhu.org/Images/Logo/BeiQiZhiZao.png', 'Tires': '225/70R16', 'SpecialTireSize': None, 'OriginalIsBaoyang': 1, 'Priority2': 1, 'Priority3': 4, 'VehicleSeries': None}


...

Traceback (most recent call last):
  File "<ipython-input-74-a1bbc79ad05d>", line 60, in <module>
    for pailiang in load_pailiang(vid):
  File "<ipython-input-74-a1bbc79ad05d>", line 11, in load_pailiang
    return requests.get(url, headers=headers).json()['PaiLiang']
KeyError: 'PaiLiang'


>> Z - 中兴 - VE-ZHON04AP - {'Vehicle': '田野皮卡-中兴汽车', 'ProductID': 'VE-ZHON04AP', 'BrandType': '中兴汽车', 'Brand': 'Z - 中兴', 'Url': '/Images/Logo/zhongxing.png', 'Src': 'https://img2.tuhu.org/Images/type/VE-ZHON04AP.png@163w_163h_100Q.png', 'IsBaoYang': 'True', 'CarName': '田野皮卡', 'appSrc': None, 'Image': 'https://img2.tuhu.org/Images/type/VE-ZHON04AP.png@100Q.png', 'ImageUrl': 'https://img2.tuhu.org/Images/Logo/ZhongXing.png', 'Tires': '215/75R15', 'SpecialTireSize': None, 'OriginalIsBaoyang': 1, 'Priority2': 1, 'Priority3': 14, 'VehicleSeries': None}


Traceback (most recent call last):
  File "<ipython-input-74-a1bbc79ad05d>", line 60, in <module>
    for pailiang in load_pailiang(vid):
  File "<ipython-input-74-a1bbc79ad05d>", line 11, in load_pailiang
    return requests.get(url, headers=headers).json()['PaiLiang']
KeyError: 'PaiLiang'


Z - 众泰 done
Z - 中欧 done
Z - 知豆 done
Z - 浙江卡尔森 done
>> Z - 浙江卡尔森 - VE-ZJKESMPV - {'Vehicle': '商务车-浙江卡尔森', 'ProductID': 'VE-ZJKESMPV', 'BrandType': '浙江卡尔森', 'Brand': 'Z - 浙江卡尔森', 'Url': '/Images/Logo/zhejiangkaersen.png', 'Src': 'https://img2.tuhu.org/Images/type/VE-ZJKESMPV.png@163w_163h_100Q.png', 'IsBaoYang': 'True', 'CarName': '商务车', 'appSrc': None, 'Image': 'https://img2.tuhu.org/Images/type/VE-ZJKESMPV.png@100Q.png', 'ImageUrl': 'https://img2.tuhu.org/Images/Logo/ZheJiangKaErSen.png', 'Tires': '205/65R16;235/60R17', 'SpecialTireSize': None, 'OriginalIsBaoyang': 1, 'Priority2': 1, 'Priority3': 1, 'VehicleSeries': None}
Z - 之诺 done


Traceback (most recent call last):
  File "<ipython-input-74-a1bbc79ad05d>", line 60, in <module>
    for pailiang in load_pailiang(vid):
  File "<ipython-input-74-a1bbc79ad05d>", line 11, in load_pailiang
    return requests.get(url, headers=headers).json()['PaiLiang']
KeyError: 'PaiLiang'


In [7]:
vid_binfo_map = {}
for _, bl in result.items():
    for binfo in bl:
        vid_binfo_map[binfo['ProductID']] = binfo


with open('./车系/child-brands-gbk.txt', '+ab') as gbk_fp, \
        open('./车系/child-brands-utf8.txt', '+ab') as utf8_fp, \
        open('./车系/child-brands-utf8-2.txt', '+ab') as utf8_fp_2, \
        open('./车系/child-brands-gbk-2.txt', '+ab') as gbk_fp_2:
        
    utf8_fp.seek(0, 0)
    for ij in utf8_fp:
        d = json.loads(ij.decode('utf8'))
        d['brand'].update(vid_binfo_map[d['vid']])
        dj = json.dumps(d, sort_keys=True, ensure_ascii=False) + '\n'
        gbk_fp_2.write(dj.encode('gbk'))
        utf8_fp_2.write(dj.encode('utf8'))
        

In [75]:
with open('child-brands-utf8.txt') as fp:
    print(fp.read().count('\n'))

89214
